# Library


In [8]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
from glob import glob
import pyexasol

In [9]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# server

In [10]:
# # insider db
wd = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='vsyse')
# exasol db
# wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
# we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')
# wbt = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='wbt')

# Load
- 등록정보&제원정보 -> result 4등급(연료 전체)
- 정기검사&정밀검사

## 등록정보(STD_CEG_CAR_MIG) 4등급만

In [12]:
# 8.6s
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0504' OR EXHST_GAS_GRD_CD = 'A05T4';")
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   VIN                0 non-null      object
 1   BSPL_STDG_CD       0 non-null      object
 2   EXHST_GAS_GRD_CD   0 non-null      object
 3   EXHST_GAS_CERT_NO  0 non-null      object
 4   VHCL_ERSR_YN       0 non-null      object
 5   MANP_MNG_NO        0 non-null      object
 6   YRIDNW             0 non-null      object
 7   VHCTY_CD           0 non-null      object
 8   PURPS_CD2          0 non-null      object
 9   FRST_REG_YMD       0 non-null      object
dtypes: object(10)
memory usage: 124.0+ bytes


In [13]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
}

In [14]:
carr = car.rename(columns=car_ch_col)

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [15]:
# 3.8s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY FROM STD_CEG_CAR_SRC_MIG;")
src.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   MANP_MNG_NO   0 non-null      object
 1   FUEL_CD       0 non-null      object
 2   VHCTY_TY_CD2  0 non-null      object
 3   MNFCTR_NM     0 non-null      object
 4   VHCNM         0 non-null      object
 5   VHCL_FRM      0 non-null      object
 6   EGIN_TY       0 non-null      object
dtypes: object(7)
memory usage: 124.0+ bytes


In [16]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
}

In [17]:
srcr = src.rename(columns=src_ch_col)

## 정기검사(STD_TB_JGT_HIS)

In [18]:
# 3m 34.9s
# jgt = wb.export_to_pandas("SELECT VIN, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_KND_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_JT_YN1, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_NLOD_SMO_PRMVL1 FROM STD_TB_JGT_HIS WHERE ROWNUM <= 10000;") # 테스트용
jgt = wd.export_to_pandas("SELECT VIN, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_KND_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_JT_YN1, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_NLOD_SMO_PRMVL1 FROM STD_TB_JGT_HIS;")
jgt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   VIN                     0 non-null      object
 1   FDRM_INSP_INSP_MTHD_CD  0 non-null      object
 2   FDRM_INSP_KND_CD        0 non-null      object
 3   FDRM_INSP_JGMT          0 non-null      object
 4   FDRM_NLOD_SMO_MSTVL1    0 non-null      object
 5   FDRM_NLOD_SMO_MSTVL2    0 non-null      object
 6   FDRM_NLOD_SMO_MSTVL3    0 non-null      object
 7   FDRM_NLOD_SMO_JT_YN1    0 non-null      object
 8   FDRM_INSP_YMD           0 non-null      object
 9   FDRM_DRVNG_DSTNC        0 non-null      object
 10  FDRM_NLOD_SMO_PRMVL1    0 non-null      object
dtypes: object(11)
memory usage: 124.0+ bytes


In [19]:
jgt_ch_col = {
    'VIN':'차대번호', 
    'FDRM_INSP_INSP_MTHD_CD':'검사방법', 
    'FDRM_INSP_KND_CD':'검사종류', 
    'FDRM_INSP_JGMT':'검사판정', 
    'FDRM_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'FDRM_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'FDRM_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'FDRM_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'FDRM_INSP_YMD':'검사일자',
    'FDRM_DRVNG_DSTNC':'주행거리',
    'FDRM_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
}

In [20]:
jgtr = jgt.rename(columns=jgt_ch_col)
jgtr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

## 정밀검사(STD_TB_EET_HIS_ME)

In [21]:
# 6m 36.1s
# eet = wb.export_to_pandas("SELECT VIN, PRCINSP_MSRMT_MTHD_CD, PRCINSP_KND_CD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PREC_NLOD_SMO_JT_YN1, PRCINSP_YMD, PRCINSP_DRVNG_DSTNC FROM STD_TB_EET_HIS_ME WHERE ROWNUM <= 10000;") # 테스트용
eet = wd.export_to_pandas("SELECT VIN, PRCINSP_MSRMT_MTHD_CD, PRCINSP_KND_CD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PREC_NLOD_SMO_JT_YN1, PRCINSP_YMD, PRCINSP_DRVNG_DSTNC, PREC_NLOD_SMO_PRMVL1 FROM STD_TB_EET_HIS_ME;")
eet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19999999 entries, 0 to 19999998
Data columns (total 11 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VIN                    object 
 1   PRCINSP_MSRMT_MTHD_CD  object 
 2   PRCINSP_KND_CD         object 
 3   PRCINSP_JGMT           object 
 4   PREC_NLOD_SMO_MSTVL1   float64
 5   PREC_NLOD_SMO_MSTVL2   float64
 6   PREC_NLOD_SMO_MSTVL3   float64
 7   PREC_NLOD_SMO_JT_YN1   object 
 8   PRCINSP_YMD            int64  
 9   PRCINSP_DRVNG_DSTNC    float64
 10  PREC_NLOD_SMO_PRMVL1   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 1.6+ GB


In [22]:
eet.columns

Index(['VIN', 'PRCINSP_MSRMT_MTHD_CD', 'PRCINSP_KND_CD', 'PRCINSP_JGMT',
       'PREC_NLOD_SMO_MSTVL1', 'PREC_NLOD_SMO_MSTVL2', 'PREC_NLOD_SMO_MSTVL3',
       'PREC_NLOD_SMO_JT_YN1', 'PRCINSP_YMD', 'PRCINSP_DRVNG_DSTNC',
       'PREC_NLOD_SMO_PRMVL1'],
      dtype='object')

In [23]:
eet_ch_col = {
    'VIN':'차대번호', 
    'PRCINSP_MSRMT_MTHD_CD':'검사방법', 
    'PRCINSP_KND_CD':'검사종류', 
    'PRCINSP_JGMT':'검사판정', 
    'PREC_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'PREC_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'PREC_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'PREC_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'PRCINSP_YMD':'검사일자',
    'PRCINSP_DRVNG_DSTNC':'주행거리', 
    'PREC_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
}

In [24]:
eetr = eet.rename(columns=eet_ch_col)
eetr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [25]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       0 non-null      object
 1   STDG_CTPV_NM  0 non-null      object
 2   STDG_SGG_NM   0 non-null      object
dtypes: object(3)
memory usage: 124.0+ bytes


In [26]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_NM', 'STDG_SGG_NM'], dtype='object')

In [27]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구',  
}

In [28]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

## RH에서 제공한 법정동코드

In [29]:
rh = we.export_to_pandas("SELECT DONG_CODE, CTPRVN_NM, SIGNGU_NM FROM TB_MAPDATA;")
rh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   DONG_CODE  278 non-null    int64 
 1   CTPRVN_NM  278 non-null    object
 2   SIGNGU_NM  260 non-null    object
dtypes: int64(1), object(2)
memory usage: 6.6+ KB


In [30]:
rh = rh.rename(columns={'DONG_CODE':'법정동코드_rh', 'CTPRVN_NM':'시도', 'SIGNGU_NM':'시군구'})
rh.head()

,법정동코드_rh,시도,시군구
0,0,전국,NaN
1,1100000000,서울특별시,NaN
2,1111000000,서울특별시,종로구
3,1114000000,서울특별시,중구
4,1117000000,서울특별시,용산구


## 4등급 result(for DPF유무)

In [31]:
# 20s
rs = we.export_to_pandas("SELECT 차대번호, DPF유무_수정 FROM GRD4_RESULT;")
rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   차대번호      1502235 non-null  object
 1   DPF유무_수정  1153813 non-null  object
dtypes: object(2)
memory usage: 22.9+ MB


# 전처리

## 중복 차대번호 제거

In [32]:
carr.shape, len(carr['차대번호'].unique())

((0, 10), 0)

In [33]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(0, 10)

## 배출가스등급 코드 변환

In [34]:
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
carr['배출가스등급'].unique()

array([], dtype=object)

## 차종 코드 변환

In [35]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array([], dtype=object)

## 용도 코드 변환

In [36]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array([], dtype=object)

## 등록정보 말소 제거

In [37]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(0, 10)

## 연료 코드 변환

In [38]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array([], dtype=object)

## 차종유형 코드 변환

In [39]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array([], dtype=object)

## 등록&제원 병합

In [40]:
# 0.7s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(0, 16)

In [41]:
cs.head()

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,차량연식,차종,용도,최초등록일자,제원관리번호,연료,차종유형,제작사명,차명,자동차형식,엔진형식


## 정기&정밀 병합

In [42]:
jgtr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

In [43]:
eetr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

In [44]:
jgtr['정기정밀'] = '정기'
eetr['정기정밀'] = '정밀'

In [45]:
# 3m 1.9s
ins = pd.concat([jgtr, eetr], ignore_index=True)
ins.shape

(19999999, 12)

### 최근 검사만 활용

In [46]:
# 4m 14.3s
insm = ins.sort_values('검사일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
insm.shape

KeyboardInterrupt: 

## 검사종류 코드 변환

In [ ]:
# 27.5s
knd_dict = {
    'A21P01':'정밀검사', 
    'A1904':'정기검사', 
    'A21T06':'종합검사(경과)', 
    'A21T05':'종합검사', 
    'A21S01':'특정검사', 
    'A1902':'재검사(2부재검사)',
    'A21R02':'2부재검사(종합)', 
    'A21P02':'정밀검사(경과)', 
    'A1903':'정기(경과)검사', 
    'A21S04':'특정재검사(배출)', 
    'A21E01':'배출재검사', 
    'A21R01':'1부재검사(종합)',
    'A21C02':'구조변경검사', 
    'A21002':'관능재검사(임시)', 
    'A21S03':'특정재검사(관능)', 
    'A21S02':'특정검사(경과)', 
    'A1901':'재검사(1부재검사)', 
    'A21T02':'종합(정밀)',
    'A21E02':'배출재검사(구변)', 
    'A21T01':'종합(경과:정밀)', 
    'A21T04':'종합(정밀) 2부재검사', 
    'A21001':'관능재검사', 
    'A21T03':'종합(정밀) 1부재검사', 
    'A21TMP':'임시검사', 
}
insm['검사종류'] = insm['검사종류'].replace(knd_dict)
insm['검사종류'].unique()

## 검사방법 코드 변환

In [ ]:
# 14.9s
mth_dict = {
    'A18A':'무부하검사(TSI)', 
    'A18B':'무부하검사(급가속)', 
    'A18C':'무부하검사(정지가동)',
    'A2301':'무부하검사(급가속)', 
    'A2302':'무부하검사(정지가동)',
    'A2303':'무부하검사(TSI)', 
    'A2304':'부하검사(LUG DOWN)', 
    'A2305':'부하검사(ASM-Idling)', 
    'A2306':'부하검사(KD-147)', 
    'A2307':'부하검사(ASM2525)', 
}
insm['검사방법'] = insm['검사방법'].replace(mth_dict)
insm['검사방법'].unique()

## 등록&제원&정기&정밀 병합

In [ ]:
# 2m 0.5s
csi = cs.merge(insm, on='차대번호', how='left')
csi.shape

In [ ]:
csi.columns

In [ ]:
csi.head()

## 등록&제원&정기&정밀&법정동코드 병합

In [ ]:
csi.shape

In [ ]:
df = csi.merge(coder, on='법정동코드', how='left')

In [ ]:
df.shape

In [ ]:
df['시도'].isnull().sum()

In [ ]:
df.loc[df['시도'].isnull(), '법정동코드']

In [ ]:
# 주소 수정
df.loc[df['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시'] # 경기도 양주시 회천3동

## rh 법정동코드 참고하여 법정동코드 수정

In [ ]:
df.columns

In [ ]:
rdf = df.copy()

In [ ]:
rdf['법정동코드'] = rdf['법정동코드'].astype('str')
rdf['법정동코드_mod'] = rdf['법정동코드'].str[:5] + '00000'
rdf['법정동코드_mod'] = pd.to_numeric(rdf['법정동코드_mod'])
rdf['법정동코드_mod'].head()

In [ ]:
set(rdf['법정동코드_mod'].unique()) - set(rh['법정동코드_rh'].unique())

In [ ]:
# 2817000000 인천광역시 남구 -> 인천광역시 미추홀구 2817700000
# 4119500000 경기도 부천시 원미구 -> 경기도 부천시 4119000000
# 4119700000 경기도 부천시 소사구 -> 경기도 부천시 4119000000
# 4119900000 경기도 부천시 오정구 -> 경기도 부천시 4119000000
# 4173000000 경기도 여주군 -> 경기도 여주시 4167000000
# 4371000000 충청북도 청원군 -> 충청북도 충주시 4311000000

In [ ]:
rdf.loc[rdf['법정동코드_mod'] == 2817000000, '법정동코드_mod'] = 2817700000
rdf.loc[rdf['법정동코드_mod'] == 4119500000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4119700000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4119900000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4173000000, '법정동코드_mod'] = 4167000000
rdf.loc[rdf['법정동코드_mod'] == 4371000000, '법정동코드_mod'] = 4311000000

In [ ]:
set(rdf['법정동코드_mod'].unique()) - set(rh['법정동코드_rh'].unique())

In [ ]:
rdf.columns

In [ ]:
rdf['법정동코드_mod'].isnull().sum()

In [ ]:
df = rdf.copy()
df.columns

## 4등급 result 파일 참고하여 DPF유무 수정

In [ ]:
rdf = df.copy()

In [ ]:
len(set(rdf['차대번호'].unique()) - set(rs['차대번호'].unique()))

In [ ]:
rs['DPF유무_수정'].value_counts(dropna=False)

In [ ]:
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rs.shape

In [ ]:
rdf1 = rdf.merge(rs, on='차대번호', how='left')

In [ ]:
rdf1['DPF유무_수정'].value_counts(dropna=False)

In [ ]:
rdf1.columns

In [ ]:
df = rdf1.copy()

In [ ]:
df.shape

# STD_BD_GRD4_CAR_CURSTT

## 4등급차만 추출

In [ ]:
# 2.8s
df1 = df[df['배출가스등급'] == '4'].reset_index(drop=True)
df1.shape

### 기준연월 정보 추가

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

In [ ]:
df1['테이블생성일자'] = today_date

In [ ]:
df1.columns

In [ ]:
STD_BD_GRD4_CAR_CURSTT = df1[[
    '테이블생성일자', 
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '용도',
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도',
    '시군구',
    '차명', 
    '제작사명', 
    '배출가스등급', 
    'DPF유무_수정',
    '검사방법', 
    '검사종류', 
    '검사판정', 
    '무부하매연측정치1', 
    '무부하매연판정1', 
    '무부하매연측정치2', 
    '무부하매연측정치3', 
    '법정동코드_mod', 
    ]]
STD_BD_GRD4_CAR_CURSTT.shape

In [ ]:
STD_BD_GRD4_CAR_CURSTT.head()

In [ ]:
ch_col_dict = {
                '테이블생성일자':'LOAD_DT',
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO', 
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2', 
                '차량연식':'YRIDNW', 
                '차종유형':'VHCTY_TY', 
                '연료':'FUEL_CD', 
                '법정동코드':'STDG_CD', 
                '시도':'CTPV_NM', 
                '시군구':'SGG_NM', 
                '차명':'VHCNM', 
                '제작사명':'MNFCTR_NM', 
                '배출가스등급':'EXHST_GAS_GRD_CD_MOD',
                'DPF유무_수정':'DPF_MNTNG_YN', 
                '검사방법':'INSP_MTHD', 
                '검사종류':'INSP_KND', 
                '검사판정':'INSP_JGMT', 
                '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
                '무부하매연판정1':'NOLOD_SMO_JGMT_YN1', 
                '무부하매연측정치2':'NOLOD_SMO_MEVLU2', 
                '무부하매연측정치3':'NOLOD_SMO_MEVLU3', 
                '법정동코드_mod':'STDG_CD_MOD',

                '제작일자':'FBCTN_YMD', 
                '차종분류':'VHCTY_CL_CD',
                }

In [ ]:
STD_BD_GRD4_CAR_CURSTT = STD_BD_GRD4_CAR_CURSTT.rename(columns=ch_col_dict)
STD_BD_GRD4_CAR_CURSTT.columns

In [ ]:
STD_BD_GRD4_CAR_CURSTT.shape

## [출력] STD_BD_GRD4_CAR_CURSTT

In [ ]:
# expdf = STD_BD_GRD4_CAR_CURSTT
# table_nm = 'STD_BD_GRD4_CAR_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 9s
# we.import_from_pandas(expdf, table_nm)

# STD_BD_GRD4_CAR_CURSTT_TOT

In [ ]:
df2 = df1.copy()

## 경유차만 추출

In [ ]:
df2 = df2[df2['연료'] == '경유'].reset_index(drop=True)

## 차대번호 10자리 연식

In [ ]:
df2['vin10'] = df2['차대번호'].str[9]

In [ ]:
vin10_dict = {'J':1988, 'K':1989, 'L':1990, 'M':1991, 'N':1992, 'P':1993, 'R':1994, 'S':1995, 'T':1996, 'V':1997, 'W':1998, 'X':1999, 'Y':2000, '1':2001, '2':2002, '3':2003, '4':2004, '5':2005, '6':2006, '7':2007, '8':2008, '9':2009, 'A':2010, 'B':2011, 'C':2012, 'D':2013, 'E':2014, 'F':2015, 'G':2016, 'H':2017}

In [ ]:
df2['vin10_year'] = df2['vin10'].map(vin10_dict, na_action='ignore')

## 배인번호_수정 문자 타입으로 변경

In [ ]:
df2['배출가스인증번호'] = df2['배출가스인증번호'].astype('str')

## 차대번호 17자리 샘플

In [ ]:
df2y = df2.loc[df2['차대번호'].str.len() == 17].reset_index(drop=True)
df2n = df2.loc[df2['차대번호'].str.len() != 17].reset_index(drop=True)
df2y.shape, df2n.shape

## 차대번호 연식과 연식 동일한 샘플

In [ ]:
df3y = df2y.loc[df2y['vin10_year'] == df2y['차량연식']].reset_index(drop=True)
df3n = df2y.loc[df2y['vin10_year'] != df2y['차량연식']].reset_index(drop=True)
df3y.shape, df3n.shape

# 배인번호별 분석

In [ ]:
def flat_cols(df):
    df.columns = ['/'.join(x) for x in df.columns.to_flat_index()]
    return df

In [ ]:
# about 31.3s
# 최적화 24m 51s -> 30.0s
total_g_df = pd.DataFrame()
groupby_col1 = ['제작사명', '배출가스인증번호', '제원관리번호', '자동차형식', '엔진형식', '검사종류', '검사방법', '검사판정']
groupby_col2 = ['제작사명', '배출가스인증번호', '제원관리번호', '자동차형식', '엔진형식', '검사종류', '검사방법']
for one in tqdm(df3y['배출가스인증번호'].unique()):
    # 배인번호별 df
    gas_df = df3y.loc[df3y['배출가스인증번호'] == str(one)].reset_index(drop=True)

    if gas_df.shape[0] != 0:
        # 제번별, 차형식별, 엔진형식별, 검사판정별 무부하매연측정치1 통계
        g = gas_df.groupby(groupby_col1).agg({'차대번호':'count', '무부하매연측정치1':['mean', 'min', 'max']}).pipe(flat_cols).round(2).reset_index()
        g = g.rename(columns={'배출가스인증번호':'배출가스인증번호', '차대번호/count':'대수', '무부하매연측정치1/mean':'mean', '무부하매연측정치1/min':'min', '무부하매연측정치1/max':'max'})
        # 하나의 배인번호에서 제번별 엔진형식별 비율 계산
        g['합격률(%)'] = round(g['대수'] / g.groupby(groupby_col2)['대수'].transform('sum') * 100, 2)
        # 종합 - 통계
        total_g_df = pd.concat([total_g_df, g], ignore_index=True)
    else:
        print(f'오류 배인번호 : {one}')
        pass

In [ ]:
total_g_df.head()

# 분석

## 열화계수(SI) 지수 계산
- SI = 측정치 / 허용치

In [ ]:
df3y.shape

In [ ]:
df3y.columns

In [ ]:
sidf = df3y[[
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '차량연식', 
    '차명', 
    '차종유형', 
    '제작사명', 
    '연료', 
    '법정동코드', 
    '배출가스인증번호', 
    '검사방법', 
    '검사종류', 
    '검사판정', 
    '주행거리', 
    '무부하매연판정1', 
    '무부하매연허용치1', 
    '무부하매연측정치1'
    ]]
sidf.shape

In [ ]:
current_yr = int(datetime.today().strftime("%Y"))
current_yr

In [ ]:
sidf['차령'] = 2023 - sidf['차량연식']
sidf['SI'] = sidf['무부하매연측정치1'] / sidf['무부하매연허용치1']

In [ ]:
sidf[['차령', 'SI']].isnull().sum()

In [ ]:
sidf[['무부하매연측정치1', '무부하매연허용치1']].isnull().sum()

In [ ]:
sidf.head()

### 경유만 추출

In [ ]:
sidf1 = sidf[sidf['연료'] == '경유'].reset_index(drop=True)

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

In [ ]:
sidf1['테이블생성일자'] = today_date

In [ ]:
sidf1 = sidf1[[
    '테이블생성일자'
    '차대번호', 
    '제원관리번호', 
    '차명', 
    '제작사명', 
    '배출가스인증번호', 
    '검사방법',
    '주행거리', 
    '차령',
    'SI', 
    ]]

In [ ]:
chc_dict = {
    '테이블생성일자':'LOAD_DT'
    '차대번호':'VIN', 
    '제원관리번호':'MANG_MNG_NO', 
    '차종':'VHCTY_CD', 
    '연식':'YRIDNW', 
    '차명':'VHCNM',
    '제작사명':'MNFCTR_NM', 
    '차종유형':'VHCTY_TY', 
    '연료':'FUEL_CD',
    '법정동코드':'STDG_CD', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
    '검사방법':'INSP_MTHD', 
    '검사종류':'INSP_KND', 
    '검사판정':'INSP_JGMT', 
    '주행거리':'DRVNG_DSTNC',
    '무부하매연판정1':'NOLOD_SMO_JGMT_YN1',
    '무부하매연허용치1':'NOLOD_SMO_PRMT_VAL1',
    '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
    '차령':'VHCAG',
    }

In [ ]:
STD_BD_GRD4_SI = sidf1.rename(columns=chc_dict)
STD_BD_GRD4_SI.columns

In [ ]:
STD_BD_GRD4_SI.head()

In [ ]:
STD_BD_GRD4_SI.shape

### [출력] SI 지수 정보(STD_BD_GRD4_SI)

In [ ]:
expdf = STD_BD_GRD4_SI
table_nm = 'STD_BD_GRD4_SI'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 6s
we.import_from_pandas(expdf, table_nm)

## 동일 배번에서 제번별 매연 boxplot

In [ ]:
total_g_df1 = total_g_df.loc[(total_g_df['배출가스인증번호'] != '확인불가') | (total_g_df['배출가스인증번호'] != 'nan')]
total_g_df1.shape

In [ ]:
sample01 = total_g_df1.loc[total_g_df1['대수'] > 100].reset_index(drop=True)
sample01.shape

### quantile

In [ ]:
# 4m 11.2s
boxplot_df = pd.DataFrame()
quantile_df = pd.DataFrame()
for one in sample01['배출가스인증번호'].unique(): # 통계에서 100대 초과인 배인번호만 추출
    temp_one = df3y.loc[(df3y['배출가스인증번호'] == one) & (df3y['검사판정'] == 'Y')].reset_index(drop=True) # 해당 배인번호 중 검사판정이 'Y'인 샘플만 추출
    if temp_one.shape[0] > 100: # 100대 초과면 진행
        for two in temp_one['검사종류'].unique(): # 검사종류별 샘플 추출
            temp_two = temp_one.loc[temp_one['검사종류'] == two].reset_index(drop=True)
            if temp_two.shape[0] > 100:
                for three in temp_two['검사방법'].unique(): # 검사방법별 샘플 추출
                    temp_three = temp_two.loc[temp_two['검사방법'] == three].reset_index(drop=True)
                    boxplot_df = pd.concat([boxplot_df, temp_three], ignore_index=True) # 해당 샘플만 추출하여 쌓기
                    if temp_three.shape[0] > 100:
                        xticks_list= []
                        data_list = []
                        for four in temp_three['제원관리번호'].unique():
                            temp_four = temp_three.loc[temp_three['제원관리번호'] == four].reset_index(drop=True).dropna(subset=['무부하매연측정치1'])
                            if temp_four.shape[0] > 100:
                                xticks_list.append(four)
                                data_list.append(temp_four['무부하매연측정치1'])
                                temp_four['q1'] = temp_four['무부하매연측정치1'].describe()['25%']
                                temp_four['q2'] = temp_four['무부하매연측정치1'].describe()['50%']
                                temp_four['q3'] = temp_four['무부하매연측정치1'].describe()['75%']
                                temp_four['차량대수'] = temp_four.shape[0]
                                quantile_df = pd.concat([quantile_df, temp_four], ignore_index=True) # 제번별 4분위 값 df형태로 저장

In [ ]:
quantile_df.shape

In [ ]:
quantile_df.columns

In [ ]:
quantile_df1 = quantile_df.drop_duplicates(['배출가스인증번호', '제원관리번호', '검사방법', '검사종류'])
quantile_df1.shape

In [ ]:
quantile_df1.head()

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

In [ ]:
quantile_df1['테이블생성일자'] = today_date

In [ ]:
quantile_df2 = quantile_df1[[
    '테이블생성일자',
    '차명',
    '제작사명', 
    '제원관리번호', 
    '배출가스인증번호', 
    '검사방법', 
    'q1', 
    'q2', 
    'q3',
    '차량대수',
    ]]
quantile_df2.shape

In [ ]:
chc_dict = {
                '테이블생성일자':'LOAD_DT', 
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO', 
                '차명':'VHCNM',
                '제작사명':'MNFCTR_NM', 
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2', 
                '차종유형':'CHCTY_TY', 
                '법정동코드':'STDG_CD', 
                '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
                '검사방법':'INSP_MTHD', 
                '검사종류':'INSP_KND', 
                '검사판정':'INSP_JGMT', 
                '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
                '무부하매연판정1':'NOLOD_SMO_JGMT_YN1',
                'q1':'LOWR_QRT',
                'q2':'MID_QRT',
                'q3':'UP_QRT',
                '차량대수':'VHCL_MKCNT',
                }
# '등급_수정':'EXHST_GAS_GRD_CD_MOD', 
# 'DPF유무_수정':'DPF_MNTNG_YN', 
# '시도명':'CTPV_NM', 
# '시군구명':'SGG_NM', 
# '차종분류':'VHCTY_CL_CD', 

In [ ]:
STD_BD_GRD4_CAR_CURSTT_TOT = quantile_df2.rename(columns=chc_dict)
STD_BD_GRD4_CAR_CURSTT_TOT.columns

In [ ]:
STD_BD_GRD4_CAR_CURSTT_TOT.shape

In [ ]:
STD_BD_GRD4_CAR_CURSTT_TOT.head()

### [출력] 제번별 4분위 값 df(STD_BD_GRD4_CAR_CURSTT_TOT)

In [ ]:
expdf = STD_BD_GRD4_CAR_CURSTT_TOT
table_nm = 'STD_BD_GRD4_CAR_CURSTT_TOT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

# code end